<a href="https://colab.research.google.com/github/KKiriri/CNN-Cifar_image-scartch-detect/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras import backend as K
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.utils import plot_model
from keras.callbacks import TensorBoard, ModelCheckpoint

# MOUNT GOOGLE DRIVE TO READ YOUR MODEL FROM GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:

batch_size = 128
epochs = 5

# input image dimensions
PATCH_SIZE = 32


def load_data():
    x = np.load('/content/gdrive/My Drive/Colab Notebooks/input.npy')
    y = np.load('/content/gdrive/My Drive/Colab Notebooks/output.npy')
    x = x.astype('float32') / 255.
    y = y.astype('float32') / 255.
    x = np.expand_dims(x, axis=-1)
    y = np.expand_dims(y, axis=-1)

    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    print(x_train.shape, x_val.shape)

    return x_train, x_val, y_train, y_val


def get_model():
    input_img = Input((PATCH_SIZE, PATCH_SIZE, 1))
    conv1 = Conv2D(32, (9,9), activation='relu', padding='same')(input_img)
    conv2 = Conv2D(64, (9,9), activation='relu', padding='same')(conv1)
    conv2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv2 = Conv2D(128, (9,9), activation='relu', padding='same')(conv2)
    upsampled_img = UpSampling2D(2)(conv2)
    conv3 = Conv2D(1, (9,9), activation='sigmoid', padding='same')(upsampled_img)

    model = Model(inputs=[input_img], outputs=[conv3])

    return model


x_train, x_val, y_train, y_val = load_data()

cnn_model = get_model()
cnn_model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy','binary_crossentropy'])

cnn_model.summary()
plot_model(cnn_model, to_file='model.png') # might need to modify
tb_callback = TensorBoard(log_dir='./Graph', histogram_freq=0, # might need to modify
                          write_graph=True, write_images=True)

checkpoint = ModelCheckpoint('/content/gdrive/My Drive/Colab Notebooks/model.h5', monitor='binary_crossentropy', verbose=1, save_best_only=True, mode='min')

cnn_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_val, y_val), callbacks=[tb_callback, checkpoint])


(209456, 32, 32, 1) (52364, 32, 32, 1)
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 32)        2624      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        165952    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       663680    
_________________________________________________________________
up_sampling2d_4 (UpSampling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_16 (Conv2D)  

# New Section